In [6]:
import sys

sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [7]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)


# geometry model
R = 0.5           # radius of the base triangle (m)
H = 1.0           # height (m)
OMEGA = 20.0      # angular velocity (rad/s)
L_BASE = R * np.sqrt(3) # length of the base triangle sides

# fixed point at origin
f1 = mss.add(Fix((0,0,0))) 

# 3 masses as the base triangle vertices
m_mass = 1.0
phi1 = 0
phi2 = 2 * np.pi / 3
phi3 = 4 * np.pi / 3

# initial positions of the 3 masses
mA = mss.add(Mass(m_mass, (R * np.cos(phi1), R * np.sin(phi1), H)))
mB = mss.add(Mass(m_mass, (R * np.cos(phi2), R * np.sin(phi2), H)))
mC = mss.add(Mass(m_mass, (R * np.cos(phi3), R * np.sin(phi3), H)))


# set initial speed
for m in mss.masses:
    # get current position 
    x, y, z = m.pos[0], m.pos[1], m.pos[2] 
    
    # seperate speed
    vx = -OMEGA * y
    vy = OMEGA * x
    vz = 0.0
    
    # input vel
    m.vel = (vx, vy, vz)


# add distance constraints
# fix-mass
L_TIP = np.sqrt(R**2 + H**2)
mss.addDistanceConstraint(DistanceConstraint(f1, mA, L_TIP))
mss.addDistanceConstraint(DistanceConstraint(f1, mB, L_TIP))
mss.addDistanceConstraint(DistanceConstraint(f1, mC, L_TIP))

# inside mass
mss.addDistanceConstraint(DistanceConstraint(mA, mB, L_BASE))
mss.addDistanceConstraint(DistanceConstraint(mB, mC, L_BASE))
mss.addDistanceConstraint(DistanceConstraint(mC, mA, L_BASE))





In [8]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []

for c in mss.constraints:
    pA = mss.fixes[c.c1.nr].pos if c.c1.type == 1 else mss.masses[c.c1.nr].pos
    pB = mss.fixes[c.c2.nr].pos if c.c2.type == 1 else mss.masses[c.c2.nr].pos

    springpos.append([pA, pB])


springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [9]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [ ]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 1)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = tuple(m.pos)

    springpos = []
    for c in mss.constraints:
        if c.c1.type == 1:                      # FIX
            pA = mss.fixes[c.c1.nr].pos
        else:                                   # MASS
            pA = mss.masses[c.c1.nr].pos

        # get position of second constraint point
        if c.c2.type == 1:
            pB = mss.fixes[c.c2.nr].pos
        else:
            pB = mss.masses[c.c2.nr].pos
        springpos.append ([ pA, pB ])
    springs.geometry = LineSegmentsGeometry(positions=springpos)

    sleep(0.02
    
    )